## Benchmark model

In this notebook a Simple Moving Average model will be created in order to have a benchmark for my DeepAR based model (to be tuned in the next notebook).

### Hyperparameters

DeepAR is the model of choice of this project.
This model expects input data to be already test-train splitted.
A big part of the model design has to be done looking close at data.
More specifically, defining these two hyperparameters about the data:
* Context length
* Prediction length

### Prediction length

This is the length of the time series future predictions in days. It will be conveniently set to 5 days (exactly a week of trading hours) because a shorter interval would be of little significance.
A longer interval could be interesting from an application point of view, but it can be challenging in terms of model performances.

### Context length

Context length can be either:
* designed on patterns or seasonality observed in the data, if any is present;
* chosen as a fixed value. This will be my choice, and it will be the same as the moving average window, in order to have a good reference metrics, applicable to both this model and the benchmark model.

To explore this second option, we will refer to what we've found during the EDA stage.

In [128]:
prediction_length = [10, 20, 50]
context_length = [10, 20, 50]

### Train, test and validation split

Time series will be all trimmed from `data_start` in `1.ExploratoryDataAnalysis.ipy`, according to DeepAR documentation, train time series should be set as the size of the entire time series less the prediction length.
Validation length will exactly equal to prediction length.

In [130]:
from data.data_utils import train_test_valid_split

#### IBM Stock train, test and validation split

In [132]:
df_ibm = df_ibm.loc[data_start:].copy()

In [134]:
df_ibm_train, df_ibm_test, df_ibm_valid = train_test_valid_split(df_ibm, prediction_length=prediction_length[1])
print(len(df_ibm), len(df_ibm_train), len(df_ibm_test), len(df_ibm_valid))

4175 4135 4155 20


In [136]:
df_ibm_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,54.979382,84.889999,85.349998,84.449997,84.750000,4704500,0.18,0.0,54.397126,54.958798,55.913001,55.525647,53.268605,56.500281,53.417314,59.046566,52.779437
2004-08-20,55.212502,85.250000,85.250000,84.519997,84.519997,4501400,0.18,0.0,54.511759,54.977556,55.852774,55.720874,53.302644,56.521944,53.433168,58.920109,52.785438
2004-08-23,54.823910,84.650002,85.449997,84.650002,85.230003,4260600,0.18,0.0,54.582999,54.969128,55.779992,55.771068,53.394930,56.515013,53.423244,58.766151,52.793832
2004-08-24,54.862782,84.709999,85.150002,84.349998,85.000000,2710400,0.18,0.0,54.564866,54.937116,55.713028,55.726906,53.402826,56.462835,53.411398,58.625949,52.800108
2004-08-25,55.095955,85.070000,85.269997,84.550003,85.000000,4405600,0.18,0.0,54.654243,54.917732,55.644653,55.829713,53.478772,56.423940,53.411524,58.447571,52.841736


In [138]:
df_ibm_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,54.979382,84.889999,85.349998,84.449997,84.750000,4704500,0.18,0.0,54.397126,54.958798,55.913001,55.525647,53.268605,56.500281,53.417314,59.046566,52.779437
2004-08-20,55.212502,85.250000,85.250000,84.519997,84.519997,4501400,0.18,0.0,54.511759,54.977556,55.852774,55.720874,53.302644,56.521944,53.433168,58.920109,52.785438
2004-08-23,54.823910,84.650002,85.449997,84.650002,85.230003,4260600,0.18,0.0,54.582999,54.969128,55.779992,55.771068,53.394930,56.515013,53.423244,58.766151,52.793832
2004-08-24,54.862782,84.709999,85.150002,84.349998,85.000000,2710400,0.18,0.0,54.564866,54.937116,55.713028,55.726906,53.402826,56.462835,53.411398,58.625949,52.800108
2004-08-25,55.095955,85.070000,85.269997,84.550003,85.000000,4405600,0.18,0.0,54.654243,54.917732,55.644653,55.829713,53.478772,56.423940,53.411524,58.447571,52.841736


In [140]:
df_ibm_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-02-22,120.860001,120.860001,121.129997,118.440002,118.500000,5879200,NaN,NaN,120.865001,119.870243,122.653422,122.934059,118.795943,123.033291,116.707195,129.125993,116.180851
2021-02-23,120.709999,120.709999,121.349998,119.800003,120.919998,4531300,NaN,NaN,120.738000,120.054926,122.565260,122.653059,118.822942,122.935035,117.174818,129.020870,116.109651
2021-02-24,123.209999,123.209999,123.489998,120.519997,120.800003,5820000,NaN,NaN,120.849001,120.171688,122.563217,123.195088,118.502913,123.364148,116.979228,129.017925,116.108508
2021-02-25,122.470001,122.470001,124.349998,122.339996,123.370003,5691800,NaN,NaN,120.872001,120.252436,122.559991,123.282327,118.461674,123.595802,116.909071,129.014722,116.105260
2021-02-26,118.930000,118.930000,122.250000,118.879997,122.250000,9044900,NaN,NaN,120.674001,120.274109,122.500570,123.377878,117.970123,123.574898,116.973320,129.034279,115.966860


#### Apple Inc. Stock train test split

In [142]:
df_aapl = df_aapl.loc[data_start:].copy()

In [144]:
df_aapl_train, df_aapl_test, df_aapl_valid = train_test_valid_split(df_aapl, prediction_length=prediction_length[1])
print(len(df_aapl), len(df_aapl_train), len(df_aapl_test), len(df_aapl_valid))

4175 4135 4155 20


In [146]:
df_aapl_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,0.472366,0.548393,0.568929,0.542143,0.562679,388920000.0,0.0,2.0,0.473627,0.481141,0.484422,0.491197,0.456057,0.504857,0.457425,0.519835,0.449008
2004-08-20,0.473750,0.550000,0.553393,0.544464,0.548393,316780800.0,0.0,2.0,0.475196,0.481218,0.484606,0.488983,0.461409,0.504821,0.457614,0.519691,0.449521
2004-08-23,0.478057,0.555000,0.558393,0.546429,0.551071,254660000.0,0.0,2.0,0.476395,0.481079,0.484711,0.488652,0.464139,0.504725,0.457433,0.519683,0.449738
2004-08-24,0.491439,0.570536,0.570536,0.556964,0.558214,374136000.0,0.0,2.0,0.477057,0.480710,0.485274,0.491797,0.462317,0.503401,0.458019,0.519742,0.450806
2004-08-25,0.508359,0.590179,0.591964,0.566607,0.569107,505618400.0,0.0,2.0,0.480195,0.481310,0.486000,0.504872,0.455517,0.506265,0.456355,0.520858,0.451141


In [148]:
df_aapl_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,0.472366,0.548393,0.568929,0.542143,0.562679,388920000.0,0.0,2.0,0.473627,0.481141,0.484422,0.491197,0.456057,0.504857,0.457425,0.519835,0.449008
2004-08-20,0.473750,0.550000,0.553393,0.544464,0.548393,316780800.0,0.0,2.0,0.475196,0.481218,0.484606,0.488983,0.461409,0.504821,0.457614,0.519691,0.449521
2004-08-23,0.478057,0.555000,0.558393,0.546429,0.551071,254660000.0,0.0,2.0,0.476395,0.481079,0.484711,0.488652,0.464139,0.504725,0.457433,0.519683,0.449738
2004-08-24,0.491439,0.570536,0.570536,0.556964,0.558214,374136000.0,0.0,2.0,0.477057,0.480710,0.485274,0.491797,0.462317,0.503401,0.458019,0.519742,0.450806
2004-08-25,0.508359,0.590179,0.591964,0.566607,0.569107,505618400.0,0.0,2.0,0.480195,0.481310,0.486000,0.504872,0.455517,0.506265,0.456355,0.520858,0.451141


In [150]:
df_aapl_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-02-22,126.000000,126.000000,129.720001,125.599998,128.009995,103916400.0,NaN,NaN,132.842000,135.049167,131.739129,139.969011,125.714989,143.693038,126.405296,141.827686,121.650572
2021-02-23,125.860001,125.860001,126.709999,118.389999,123.760002,158273000.0,NaN,NaN,131.737000,134.206829,131.824364,139.462184,124.011815,142.990963,125.422695,141.631303,122.017424
2021-02-24,125.349998,125.349998,125.559998,122.230003,124.940002,111039900.0,NaN,NaN,130.671000,133.327009,131.870241,138.711130,122.630870,141.948849,124.705169,141.530117,122.210365
2021-02-25,120.989998,120.989998,126.459999,120.540001,124.680000,148199500.0,NaN,NaN,129.231000,132.284107,131.845494,138.568264,119.893735,141.585354,122.982861,141.611953,122.079035
2021-02-26,121.260002,121.260002,124.849998,121.199997,122.589996,164320000.0,NaN,NaN,127.843999,131.502835,131.838728,137.404706,118.283293,141.753261,121.252408,141.634587,122.042870


#### Amazon Stock train test split

In [151]:
df_amzn = df_amzn.loc[data_start:].copy()

In [152]:
df_amzn_train, df_amzn_test, df_amzn_valid = train_test_valid_split(df_amzn, prediction_length=prediction_length[1])
print(len(df_amzn), len(df_amzn_train), len(df_amzn_test), len(df_amzn_valid))

4175 4135 4155 20


In [153]:
df_amzn_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,38.630001,38.630001,40.480000,36.349998,40.259998,12696100.0,NaN,NaN,37.106001,37.6520,44.9828,39.862539,34.349462,40.417963,34.886038,57.693587,32.272014
2004-08-20,39.509998,39.509998,39.910000,38.110001,38.459999,6790800.0,NaN,NaN,37.508000,37.6285,44.7682,40.386916,34.629084,40.318115,34.938885,57.479021,32.057379
2004-08-23,39.450001,39.450001,40.000000,39.110001,39.889999,5532600.0,NaN,NaN,37.921000,37.6620,44.5584,40.581550,35.260450,40.427620,34.896380,57.267085,31.849715
2004-08-24,39.049999,39.049999,39.930000,38.320000,39.720001,7640400.0,NaN,NaN,38.116000,37.6590,44.3544,40.794891,35.437110,40.418129,34.899871,57.083144,31.625656
2004-08-25,40.299999,40.299999,40.490002,38.160000,39.060001,7254800.0,NaN,NaN,38.490000,37.7755,44.1582,41.246567,35.733433,40.776113,34.774887,56.827117,31.489283


In [154]:
df_amzn_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,38.630001,38.630001,40.480000,36.349998,40.259998,12696100.0,NaN,NaN,37.106001,37.6520,44.9828,39.862539,34.349462,40.417963,34.886038,57.693587,32.272014
2004-08-20,39.509998,39.509998,39.910000,38.110001,38.459999,6790800.0,NaN,NaN,37.508000,37.6285,44.7682,40.386916,34.629084,40.318115,34.938885,57.479021,32.057379
2004-08-23,39.450001,39.450001,40.000000,39.110001,39.889999,5532600.0,NaN,NaN,37.921000,37.6620,44.5584,40.581550,35.260450,40.427620,34.896380,57.267085,31.849715
2004-08-24,39.049999,39.049999,39.930000,38.320000,39.720001,7640400.0,NaN,NaN,38.116000,37.6590,44.3544,40.794891,35.437110,40.418129,34.899871,57.083144,31.625656
2004-08-25,40.299999,40.299999,40.490002,38.160000,39.060001,7254800.0,NaN,NaN,38.490000,37.7755,44.1582,41.246567,35.733433,40.776113,34.774887,56.827117,31.489283


In [155]:
df_amzn_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-02-22,3180.739990,3180.739990,3232.320068,3172.260010,3208.129883,3515700.0,NaN,NaN,3279.081958,3290.295471,3231.051772,3365.634028,3192.529888,3392.252777,3188.338165,3386.301978,3075.801567
2021-02-23,3194.500000,3194.500000,3204.729980,3093.600098,3127.030029,4677200.0,NaN,NaN,3266.237964,3285.320471,3232.857773,3361.541549,3170.934379,3395.865222,3174.775720,3384.134985,3081.580562
2021-02-24,3159.530029,3159.530029,3171.229980,3125.379883,3166.750000,3011300.0,NaN,NaN,3251.690967,3276.990479,3234.018574,3363.651628,3139.730306,3399.091150,3154.889808,3382.037003,3086.000145
2021-02-25,3057.159912,3057.159912,3178.260010,3047.760010,3136.739990,4533800.0,NaN,NaN,3228.748950,3268.219470,3232.833374,3391.456454,3066.041447,3424.242619,3112.196322,3385.569009,3080.097739
2021-02-26,3092.929932,3092.929932,3122.439941,3036.699951,3095.199951,4273500.0,NaN,NaN,3211.828955,3260.984961,3231.552573,3393.225455,3030.432455,3435.325008,3086.644913,3387.915790,3075.189357


#### Alphabet Inc. Stock train test split

In [156]:
df_googl = df_googl.loc[data_start:].copy()

In [157]:
df_googl_train, df_googl_test, df_googl_valid = train_test_valid_split(df_googl, prediction_length=prediction_length[1])
print(len(df_googl), len(df_googl_train), len(df_googl_test), len(df_googl_valid))

4175 4135 4155 20


In [158]:
df_googl_train.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,50.220219,50.220219,52.082081,48.028027,50.050049,44659096.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,54.209209,54.209209,54.594597,50.300301,50.555557,22834343.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,54.754753,54.754753,56.796799,54.579578,55.430431,18256126.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,52.487488,52.487488,55.855858,51.836838,55.675674,15247337.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,53.053055,53.053055,54.054054,51.991993,52.532532,9188602.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [159]:
df_googl_test.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2004-08-19,50.220219,50.220219,52.082081,48.028027,50.050049,44659096.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-20,54.209209,54.209209,54.594597,50.300301,50.555557,22834343.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-23,54.754753,54.754753,56.796799,54.579578,55.430431,18256126.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-24,52.487488,52.487488,55.855858,51.836838,55.675674,15247337.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381
2004-08-25,53.053055,53.053055,54.054054,51.991993,52.532532,9188602.0,0.0,1.998,52.432933,52.692943,63.586987,55.781053,49.084813,56.933168,48.452718,88.331593,38.842381


In [160]:
df_googl_valid.head()

,Adj Close,Close,High,Low,Open,Volume,Dividends,Stock Splits,10_ac_ma,20_ac_ma,50_ac_ma,10_ac_bb_u,10_ac_bb_l,20_ac_bb_u,20_ac_bb_l,50_ac_bb_u,50_ac_bb_l
Date,,,,,,,,,,,,,,,,,
2021-02-22,2054.260010,2054.260010,2080.330078,2048.000000,2053.600098,1473300.0,NaN,NaN,2090.837012,2011.181488,1862.419397,2127.667275,2054.006748,2225.933273,1796.429703,2150.760197,1574.078597
2021-02-23,2060.120117,2060.120117,2072.500000,1990.229980,2013.989990,1732900.0,NaN,NaN,2088.397021,2019.473492,1868.064600,2130.009741,2046.784302,2227.934498,1811.012487,2160.668947,1575.460252
2021-02-24,2083.810059,2083.810059,2088.320068,2027.530029,2030.040039,1130400.0,NaN,NaN,2089.239038,2028.266498,1874.387800,2130.014337,2048.463739,2231.695684,1824.837312,2171.760767,1577.014833
2021-02-25,2015.949951,2015.949951,2083.000000,2006.489990,2055.929932,2077400.0,NaN,NaN,2082.186035,2038.116998,1879.210798,2144.035802,2020.336269,2216.392216,1859.841780,2177.810921,1580.610676
2021-02-26,2021.910034,2021.910034,2056.520020,2001.010010,2029.949951,2001500.0,NaN,NaN,2075.502039,2046.552502,1884.603799,2147.768484,2003.235593,2202.561915,1890.543090,2183.585370,1585.622228


### Metrics computation

Computing metrics on the benchmark model will give me a good reference to evaluate the deep learning model after training.

As can be presumed and seen from data prints, first `n` values of validation data are `NaN` so, it will be better just to exclude the first `n-1` values from the validation set to avoid misleading values to be catch in metrics evaluation. 

In [161]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score

#### IBM stock

In [162]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [163]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid.iloc[n-1:][ma_str])

In [164]:
#print(ibm_ma_mse_loss)

In [165]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [166]:
ibm_ma_mae_loss = mean_absolute_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [167]:
print(ibm_ma_mae_loss)

3.1680560684204098


Root Mean Squared Error

In [168]:
ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str], squared=False)

In [169]:
print(ibm_ma_mse_loss)

3.8068639837005347


Mean Absolute Percentage Error

In [170]:
ibm_ma_map_loss = mean_absolute_percentage_error(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [171]:
print(ibm_ma_map_loss)

0.025014186682152002


R<sup>2</sup> score

In [172]:
ibm_ma_r2_score = r2_score(df_ibm_valid.iloc[:]['Adj Close'], df_ibm_valid[:][ma_str])

In [173]:
print(ibm_ma_r2_score)

-0.14943118195926175


In [174]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [175]:
#ibm_ma_mse_loss = mean_squared_error(df_ibm_valid.iloc[n-1:]['Adj Close'], df_ibm_valid[n-1:][ma_str])

In [176]:
#print(ibm_ma_mse_loss)

#### Apple Inc. stock

In [177]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [178]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid.iloc[n-1:][ma_str])

In [179]:
#print(aapl_ma_mse_loss)

In [180]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [181]:
aapl_ma_mae_loss = mean_absolute_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [182]:
print(aapl_ma_mae_loss)

6.002291526794432


##### Root Mean Squared Error

In [183]:
aapl_ma_rmse_loss = mean_squared_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str], squared=False)

In [184]:
print(aapl_ma_rmse_loss)

6.884638216801444


##### Mean Absolute Percentage Error

In [185]:
aapl_ma_map_loss = mean_absolute_percentage_error(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [186]:
print(aapl_ma_map_loss)

0.04919876190165914


R<sup>2</sup> score

In [187]:
aapl_ma_r2_score = r2_score(df_aapl_valid.iloc[:]['Adj Close'], df_aapl_valid[:][ma_str])

In [188]:
print(aapl_ma_r2_score)

-5.209950329704853


In [189]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [190]:
#aapl_ma_mse_loss = mean_squared_error(df_aapl_valid.iloc[n-1:]['Adj Close'], df_aapl_valid[n-1:][ma_str])

In [191]:
#print(aapl_ma_mse_loss)

#### Amazon.com stock

In [192]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [193]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid.iloc[n-1:][ma_str])

In [194]:
#print(amzn_ma_mse_loss)

In [195]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [196]:
amzn_ma_mae_loss = mean_absolute_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [197]:
print(amzn_ma_mae_loss)

112.0814318847656


Root Mean Squared Error

In [198]:
amzn_ma_rmse_loss = mean_squared_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str], squared=False)

In [199]:
print(amzn_ma_rmse_loss)

133.9031253934699


##### Mean Absolute Percentage Error

In [200]:
amzn_ma_map_loss = mean_absolute_percentage_error(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [201]:
print(amzn_ma_map_loss)

0.0366753970702243


R<sup>2</sup> score

In [202]:
amzn_ma_r2_score = r2_score(df_amzn_valid.iloc[:]['Adj Close'], df_amzn_valid[:][ma_str])

In [203]:
print(amzn_ma_r2_score)

-3.389770931060588


In [204]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [205]:
#amzn_ma_mse_loss = mean_squared_error(df_amzn_valid.iloc[n-1:]['Adj Close'], df_amzn_valid[n-1:][ma_str])

In [206]:
#print(amzn_ma_mse_loss)

#### Alphabet Inc. stock

In [207]:
#n = 10
#ma_str = str(n)+'_ac_ma'

In [208]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid.iloc[n-1:][ma_str])

In [209]:
#print(googl_ma_mse_loss)

In [210]:
n = 20
ma_str = str(n)+'_ac_ma'

Mean Absolute Error

In [211]:
googl_ma_mae_loss = mean_absolute_error(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [212]:
print(googl_ma_mae_loss)

29.841813964843766


Root Mean Squared Error

In [213]:
googl_ma_rmse_loss = mean_squared_error(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str], squared=False)

In [214]:
print(googl_ma_rmse_loss)

33.8341527880325


##### Mean Absolute Percentage Error

In [215]:
googl_ma_map_loss = mean_absolute_percentage_error(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [216]:
print(googl_ma_map_loss)

0.014581975406481384


R<sup>2</sup> score

In [217]:
googl_ma_r2_score = r2_score(df_googl_valid.iloc[:]['Adj Close'], df_googl_valid[:][ma_str])

In [218]:
print(googl_ma_r2_score)

-0.4344092626188014


In [219]:
#n = 50
#ma_str = str(n)+'_ac_ma'

In [220]:
#googl_ma_mse_loss = mean_squared_error(df_googl_valid.iloc[n-1:]['Adj Close'], df_googl_valid[n-1:][ma_str])

In [221]:
#print(googl_ma_mse_loss)

Volatility

In [222]:
print(volatility(df_ibm_valid['Adj Close'], n))

0.6635874205871215


In [223]:
print(volatility(df_aapl_valid['Adj Close'], n))

0.40171728528815365


In [224]:
print(volatility(df_amzn_valid['Adj Close'], n))

214.97401538980654


In [225]:
print(volatility(df_googl_valid['Adj Close'], n))

42.00335011611448


As expected, loss augments as we observe moving average on larger windows.
Also, we can observe that Amazon.com and Alphabet Inc. have greater losses, that also corresponds to higher volatility.